In [1]:
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torch


class AbaloneDataset(Dataset):
    """ Diabetes dataset.""" 
    # Initialize your data, download, etc. 
    def __init__(self, train): 
        import pandas as pd
        
        df = pd.read_csv('data/chap01/abalone.csv', header=None, skiprows=1)

        input_cnt, output_cnt = 10, 1
        data = np.zeros([len(df), input_cnt + output_cnt])

        # 원래있던 sex칼럼을 원핫 인코딩을 적용하여 3 칼럼으로 만들고 나머지 칼럼을 복사해온다.
        for index, row in df.iterrows():
            if row[0] == 'I':
                data[index, 0] = 1
            if row[0] == 'M':
                data[index, 1] = 1
            if row[0] == 'F':
                data[index, 2] = 1

            data[:, 3:] = df.loc[:, 1:]
            
        data.astype('float')
        
        if train is True:
            data = data[:int(len(data)*0.8),:]
        else:
            data = data[int(len(data)*0.8):,:]
        
        self.len = data.shape[0] 
        self.x_data = torch.from_numpy(data[:, 0:-1]).float() 
        self.y_data = torch.from_numpy(data[:, [-1]]).float()
        
    def __getitem__(self, index): 
        return self.x_data[index], self.y_data[index] 
    
    def __len__(self): 
        return self.len


In [2]:
train_loader = DataLoader(
    dataset     = AbaloneDataset(1),
    batch_size  = 32,
    shuffle     = True,
    num_workers = 4)

test_loader = DataLoader(
    dataset     = AbaloneDataset(0),
    batch_size  = 32,
    shuffle     = True)

In [3]:
class NeuralNet(torch.nn.Module):
    def __init__(self, input_size, hidden_size):
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        
        self.linear_1 = torch.nn.Linear(self.input_size, self.hidden_size, bias=True)
        
    def forward(self, input_tensor):
        linear1 = self.linear_1(input_tensor)
        
        return linear1
        

In [4]:
def train(model, train_loader, optimizer):
    model.train()
    for batch_idx, (train_data, target) in enumerate(train_loader):
        # 학습 데이터를 DEVICE의 메모리로 보냄
        #print(DEVICE)
        data, target = train_data.to(DEVICE), target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for test_data, target in test_loader:
            #print(DEVICE)
            test_data, target = test_data.to(DEVICE), target.to(DEVICE)
            output = model(test_data)

            # 모든 오차 더하기
            test_loss += criterion(output, target).item()
            

    test_loss /= len(test_loader.dataset)
    return test_loss



torch.set_default_tensor_type(torch.cuda.FloatTensor)

In [5]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
#DEVICE = 'cpu'

In [6]:
EPOCHS = 30
BATCH_SIZE = 64

In [7]:
model        = NeuralNet(10, 1).to(DEVICE)
optimizer    = torch.optim.SGD(model.parameters(), lr=0.01)
criterion    = torch.nn.MSELoss() 

In [8]:
print(model)

NeuralNet(
  (linear_1): Linear(in_features=10, out_features=1, bias=True)
)


In [12]:
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer)
    test_loss = evaluate(model, test_loader)
    
    print('[{}] Test Loss: {:.4f}'.format(
          epoch, test_loss))

[1] Test Loss: 0.1592
[2] Test Loss: 0.1503
[3] Test Loss: 0.1527
[4] Test Loss: 0.1498
[5] Test Loss: 0.1497
[6] Test Loss: 0.1542
[7] Test Loss: 0.1520
[8] Test Loss: 0.1588
[9] Test Loss: 0.1528
[10] Test Loss: 0.1497
[11] Test Loss: 0.1486
[12] Test Loss: 0.1509
[13] Test Loss: 0.1484
[14] Test Loss: 0.1489
[15] Test Loss: 0.1478
[16] Test Loss: 0.1764
[17] Test Loss: 0.1550
[18] Test Loss: 0.1491
[19] Test Loss: 0.1488
[20] Test Loss: 0.1492
[21] Test Loss: 0.1480
[22] Test Loss: 0.1478
[23] Test Loss: 0.1500
[24] Test Loss: 0.1489
[25] Test Loss: 0.1505
[26] Test Loss: 0.1492
[27] Test Loss: 0.1544
[28] Test Loss: 0.1674
[29] Test Loss: 0.1482
[30] Test Loss: 0.1469
